# CS 4452 Assignment 1

## Problem 1: Image Detection

##### 1. Smoothing Image 1 

using a 5×5 Gaussian filter with σ = 1 and σ = 2 (Implement the convolution process). Plot both smoothed images.

#### 2. Apply Filters & Plotting 

implementing 3×3 Sobel filters 𝑆𝑥and 𝑆𝑦to each smoothed image from (a). Plot the
resulting 𝐺𝑥and 𝐺𝑦images (and optionally the gradient magnitude).

#### 3. Convolving Image 1 

using derivative-of-Gaussian filters ∂𝐺/ ∂𝑥 and ∂𝐺/ ∂𝑦, using
5×5 filters for σ = 1 and σ = 2. Plot the resulting 𝐺𝑥and 𝐺𝑦images for each σ

#### 4. Comparision 

the results of (b) and (c). Describing what is observed and why.

## Problem 2: Corner Detection

#### 1. Harris Corner Detector 

Implementing step-by-step and applying it to Image 1 and provide:

 - the corner response map 𝑅, and

 - the output after non-maximum suppression (final detected corners, ideally
overlaid on the image). 